### 1. Extracting Our Data from an API

Now Max's Wine Dive is a restaurant with multiple locations in Texas.

Let's try to see what information we can find on the Max's by using the Texas's Open Data API. Navigating to the Mixed Beverage Receipts data we see that we can search for specific restaurants using the location_name parameter. Let's do that below

In [152]:
url = "https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE"

Use the url above to make a request to the API, and store the json results as restaurant_receipts.

In [153]:
import requests
response = requests.get(url)
restaurant_receipts = response.json()

In [154]:
len(restaurant_receipts)

61

### 2. Understanding What's Returned

In [330]:
first_receipt = restaurant_receipts[0]
first_receipt

{'taxpayer_number': '12727298569',
 'taxpayer_name': 'MWD AUSTIN DOWNTOWN, LLC',
 'taxpayer_address': '7026 OLD KATY RD STE 255',
 'taxpayer_city': 'HOUSTON',
 'taxpayer_state': 'TX',
 'taxpayer_zip': '77024',
 'taxpayer_county': '101',
 'location_number': '1',
 'location_name': "MAX'S WINE DIVE",
 'location_address': '207 SAN JACINTO BLVD STE 200',
 'location_city': 'AUSTIN',
 'location_state': 'TX',
 'location_zip': '78701',
 'location_county': '227',
 'inside_outside_city_limits_code_y_n': 'Y',
 'tabc_permit_number': 'MB944126',
 'responsibility_begin_date_yyyymmdd': '2016-05-13T00:00:00.000',
 'obligation_end_date_yyyymmdd': '2016-09-30T00:00:00.000',
 'liquor_receipts': '18265',
 'wine_receipts': '71497',
 'beer_receipts': '10606',
 'cover_charge_receipts': '0',
 'total_receipts': '100368'}

In [327]:
# creates list of all location addresses
location_addresses = []

for rest_data in restaurant_receipts:
    address = rest_data['location_address']
    location_addresses.append(address)

In [328]:
set(location_addresses)

{'207 SAN JACINTO BLVD STE 200', '3600 MCKINNEY AVE STE 100'}

In [329]:
# creates list of dict of all location addresses
location_addresses = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE&$select=location_address").json()
#location_addresses

***

### 3. Reduce Our Data (Transform)

#### A. Reduce the number of items

In [325]:
dallas_maxs = requests.get("https://data.texas.gov/resource/naix-2893.json?location_address=3600 MCKINNEY AVE STE 100").json()

In [331]:
dallas_maxs[0]['location_address']

'3600 MCKINNEY AVE STE 100'

#### B. Reduce the amount of data per item

Ok, so now that we have cut the number of items in half, let's also limit the amount of information in each item. Let's start by remembering what information contained in each dictionary. An easy way to do so is to use the keys method on our dictionary.

In [177]:
first_dallas_receipt = dallas_maxs[0]
first_dallas_receipt.keys()

dict_keys(['taxpayer_number', 'taxpayer_name', 'taxpayer_address', 'taxpayer_city', 'taxpayer_state', 'taxpayer_zip', 'taxpayer_county', 'location_number', 'location_name', 'location_address', 'location_city', 'location_state', 'location_zip', 'location_county', 'inside_outside_city_limits_code_y_n', 'tabc_permit_number', 'responsibility_begin_date_yyyymmdd', 'responsibility_end_date_yyyymmdd', 'obligation_end_date_yyyymmdd', 'liquor_receipts', 'wine_receipts', 'beer_receipts', 'cover_charge_receipts', 'total_receipts'])

Ok, so we can see that a lot of information is here. Let's reduce our information by only including the total_receipts, the responsibility_begin_date_yyyymmdd and the obligation_end_date_yyyymmdd.

In [335]:
# reduce information to only include the 'total_receipts', 'responsibility_begin_date_yyyymmdd' and 'obligation_end_date_yyyymmdd'
restaurant_revenues = requests.get("https://data.texas.gov/resource/naix-2893.json?location_address=3600 MCKINNEY AVE STE 100&$select=total_receipts,responsibility_begin_date_yyyymmdd,obligation_end_date_yyyymmdd").json()

len(restaurant_revenues)


25

In [336]:
restaurant_revenues[0:2]

[{'total_receipts': '56182',
  'responsibility_begin_date_yyyymmdd': '2015-08-11T00:00:00.000',
  'obligation_end_date_yyyymmdd': '2016-12-31T00:00:00.000'},
 {'total_receipts': '9400',
  'responsibility_begin_date_yyyymmdd': '2015-08-11T00:00:00.000',
  'obligation_end_date_yyyymmdd': '2017-08-31T00:00:00.000'}]

Now looking at the first two elements we see that the end date seems to proceed monthly but that begin_date is always May 13 2016. It seems like this just marks the first time that Max's needed to submit information. We don't need to know this, so let's remove it from our list.

In [337]:
revenues_by_date = requests.get("https://data.texas.gov/resource/naix-2893.json?location_address=3600 MCKINNEY AVE STE 100&$select=total_receipts,obligation_end_date_yyyymmdd").json()


len(revenues_by_date)
# 25

25

In [338]:
revenues_by_date[0:2]

[{'total_receipts': '56182',
  'obligation_end_date_yyyymmdd': '2016-12-31T00:00:00.000'},
 {'total_receipts': '9400',
  'obligation_end_date_yyyymmdd': '2017-08-31T00:00:00.000'}]

***

### 4. Coerce the data (Still Transform)

Our final step will be to coerce our data to the correct format. We'd like total receipts to be an integer and we'd like the begin date to be of type datetime.

This is a little tricky so let's do it with dictionary first object first.

In [343]:
first_rev_by_date = revenues_by_date[0]
first_rev_by_date

{'total_receipts': '56182',
 'obligation_end_date_yyyymmdd': '2016-12-31T00:00:00.000'}

We can go from a string to an integer by using the int function. The int function is called a constructor because it's used to construct integers. We can use it so long as we pass in a string that can be changed to an integer.

In [344]:
int(first_rev_by_date['total_receipts'])

56182

Ok, now let's coerce the string into a datetime. First we ask the great oracle Google how we can convert a string into a datetime. Then we follow the directions in the search results. One of those results says we can convert with somthing like the following.

In [341]:
from datetime import datetime
start = '2011-01-03'
datetime.strptime(start, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

That gets us part of the way, but our date information includes information about minutes or seconds. So we can either remove that ending data, or we can do some more searching on Google. We go for the searching on Google and wind up with the following.

In [346]:
from datetime import datetime
end_date = first_rev_by_date['obligation_end_date_yyyymmdd']
datetime.strptime(end_date, '%Y-%m-%dT%H:%M:%S.%f')

datetime.datetime(2016, 12, 31, 0, 0)

Ok, now it's that we were able to accomplish this for the attributes of one dictionary, let's use a loop to coerce each dictionary in revenues_by_date.

In [348]:
from datetime import datetime
formatted_revenues = []
for revenue in revenues_by_date:
    total = int(revenue['total_receipts'])
    revenue = {
        'total_receipts': total,
        'end_date': revenue['obligation_end_date_yyyymmdd'],
    }
    formatted_revenues.append(revenue)
formatted_revenues[0:2]

[{'total_receipts': 56182, 'end_date': '2016-12-31T00:00:00.000'},
 {'total_receipts': 9400, 'end_date': '2017-08-31T00:00:00.000'}]

### 5. Store our data (Load)

Once we have our date in a good format. Let's store that data so that we can use it in some future research. The following code is slightly confusing, but it's also freely available on the Internet. So let's use it to write our data to a file.

In [349]:
import json
with open('maxs_revenues.json', 'w') as filehandle:  
    json.dump(formatted_revenues, filehandle)

We can eaily check that we stored this data correctly by attempting to read that data.

In [350]:
with open('maxs_revenues.json') as json_file:  
    pulled_revenues = json.load(json_file)


In [351]:
len(pulled_revenues)

25

In [352]:
pulled_revenues[0:2]

[{'total_receipts': 56182, 'end_date': '2016-12-31T00:00:00.000'},
 {'total_receipts': 9400, 'end_date': '2017-08-31T00:00:00.000'}]